Задание 6. Задача об оптимальном использовании ресурсов

Для производства трех видов изделий (A, B, C) используются ресурсы типа I и II,
причем закупки ресурсов ограничены возможностями поставщиков. Нормы расхода ресурсов и их запасы приведены в таблице.

1. Постройте математическую модель задачи.

2. Определите такой план производства, при котором стоимость произведенного товара из имеющихся ресурсов является наибольшей.

3. Постройте задачу, двойственную к данной.

4. Найдите решение двойственной задачи. Поясните смысл двойственных переменных.

5. Проверьте результаты вычислений на компьютере.

Вариант 9

<b>Математическая модель задачи.</b><br>
Необходимо максимизировать функцию<br>
*F(x1,x2,x3) = (6b + 12)\*x1 + (5b + 22)\*x2 + c\*x3* → max<br>
при заданных ограничениях ресурсов:<br>
*x1 + 3*\*x2 + a*\*x3 <= 3000*<br>
6*\*x1 + 5*\*x2 + 2*\*x3 <= 3320*<br>
*x1, x2, x3 >= 0*

In [17]:
import numpy as np
from scipy.optimize import Bounds, linprog
from gurobipy import *

In [18]:
a = int(input('a = '))
b = int(input('b = '))
c = int(input('c = '))

#### Находим оптимальный план производства

1. используя linprog

In [19]:
# границы и ограничения

bounds = np.array([(0, np.inf), (0, np.inf), (0, np.inf)])

A_ub = np.array([[1, 3, a], [6, 5, 2]])
b_ub = np.array([3000, 3320])
c0 = np.array([6*b + 12, 5*b + 22, c])

res = linprog(-c0, A_ub, b_ub, bounds=bounds, method='simplex', options={'disp': True})

x1 = res.x[0]
x2 = res.x[1]
x3 = res.x[2]
F_max = -res.fun
print(res.x)

Optimization terminated successfully.
         Current function value: -28877.500000
         Iterations: 3
[247.5   0.  917.5]


In [20]:
print(f'Оптимальные значения: x1 = {x1.round(3)}, x2 = {x2.round(3)}, x3 = {x3.round(3)}')
print('Max значение функции:',F_max.round(3))

Оптимальные значения: x1 = 247.5, x2 = 0.0, x3 = 917.5
Max значение функции: 28877.5


In [21]:
b_opt = np.array([x1 + 3*x2 + a*x3, 6*x1 + 5*x2 + 2*x3]) # Реальный расход ресурсов
b_opt

array([3000., 3320.])

In [22]:
#Дефицит
abs(b_ub-b_opt)

array([0., 0.])

2. используя gurobi

In [23]:
var_names = ['x1', 'x2', 'x3']                         
profit = np.array([6*b + 12, 5*b + 22, c])
res_used = np.array([[1, 3, a], [6, 5, 2]])
res_avail = np.array([3000, 3320])

n = len(profit)                                    
m = len(res_avail)                                 
model = Model()
x = model.addVars(n, name=var_names)
obj = model.setObjective(quicksum(profit[j] * x[j] for j in range(n)), GRB.MAXIMIZE)
con = []
for i in range(m):
    con.append(model.addConstr(quicksum(res_used[i, j] * x[j] for j in range(n)) <= res_avail[i], name=var_names[i]))
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0xb9347198
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+03, 3e+03]
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.1000000e+31   3.375000e+30   8.100000e+01      0s
       2    2.8877500e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.887750000e+04


In [24]:
# вывод параметров
print("X =", model.X, "\nFmax =", model.ObjVal, "\nОптимальный план двойственной задачи Pi → (y1,y2) =", model.Pi) 
print("x1 =", x[0].X)
print("Pi1 =", con[0].Pi)

X = [247.5, 0.0, 917.5] 
Fmax = 28877.5 
Оптимальный план двойственной задачи Pi → (y1,y2) = [6.375000000000001, 2.9375]
x1 = 247.5
Pi1 = 6.375000000000001


In [25]:
model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])


    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
          x1        247.5           24      20.7778           75 
          x2            0           32         -inf      33.8125 
          x3        917.5           25      22.7692           72 


In [26]:
print('Анализ чувствительности\nFmax =', model.ObjVal)
model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])
model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp'])




Анализ чувствительности
Fmax = 28877.5

    Variable            X           RC           LB      SALBLow       SALBUp           UB      SAUBLow       SAUBUp 
--------------------------------------------------------------------------------------------------------------------
          x1        247.5            0            0         -inf        247.5          inf        247.5          inf 
          x2            0      -1.8125            0         -inf          440          inf            0          inf 
          x3        917.5            0            0         -inf        917.5          inf        917.5          inf 

  Constraint        Sense        Slack           Pi          RHS     SARHSLow      SARHSUp 
------------------------------------------------------------------------------------------
          x1            <            0        6.375         3000      553.333         4980 
          x2            <            0       2.9375         3320         2000        18000 
